<a href="https://colab.research.google.com/github/chandradutta/DL_Assignment_2/blob/main/Part_B/DL_Assignment2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchvision.models as models
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,Subset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torch.nn.functional as func
# import wandb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader
import torchvision

In [ ]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip

In [ ]:
!rm nature_12K.zip

In [ ]:
dataset = torchvision.datasets.ImageFolder(root='inaturalist_12K/train', transform=transforms)
from torchvision import transforms

transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229, 0.224, 0.225])])

train_dataset = ImageFolder(root='inaturalist_12K/train', transform=transform)
# test_dataset = ImageFolder(root='/kaggle/input/inaturalist-dataset/inaturalist_12K/val', transform=transform)


# Define the size of the validation set (20%)
validation_size = 0.2

# Split the dataset into training and validation sets with shuffling
train_data, val_data = train_test_split(list(range(len(train_dataset))), test_size=validation_size, shuffle=True, random_state=42)
#Data Loader

train_subset = Subset(train_dataset, train_data)
val_subset  = Subset(train_dataset, val_data)


In [ ]:
from tqdm import tqdm

In [ ]:
def DataLoaders(aug,batch_size):
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=True)
    return train_loader,val_loader

In [ ]:
def RESNET50(k,NUM_OF_CLASSES): #this function returns the model by freezing first k layers only
    model = models.resnet50(pretrained=True)

    params = list(model.parameters())
    for param in params[:k]:
        param.requires_grad = False #freezing

    num_ftrs = model.fc.in_features

    model.fc = torch.nn.Linear(num_ftrs, NUM_OF_CLASSES)

    return model

In [ ]:
LossFun = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # integrating with gpu

def train_model(epochs,activationFun,aug,batch_size,eta):
    train_loader,val_loader = DataLoaders(aug,batch_size)
    Network = RESNET50(50,10).to(device)
    Algo = torch.optim.Adam(Network.parameters(), lr=eta)
    for epoch in range(epochs):
        for i,(images,labels) in enumerate(tqdm(train_loader)):

            images = images.to(device)
            labels = labels.to(device)
            # Forward pass
            outputs = Network(images)
            loss = LossFun(outputs, labels)

            # Backward and optimize
            Network.zero_grad()
            loss.backward()
            Algo.step()
        train_acc,train_loss = calculate_accuracy(Network,train_loader,activationFun)
        Validation_acc,val_loss=calculate_accuracy(Network, val_loader,activationFun)
        # wandb.log({"Validation_acc" : Validation_acc})
        # wandb.log({"val_loss" : val_loss})
        # wandb.log({"train_acc" : train_acc})
        # wandb.log({"train_loss" : train_loss})
        print(train_acc,train_loss)
        print(Validation_acc,val_loss)


#accuracy
def calculate_accuracy(model, data_loader,activationFun):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    num_loss=0
    total_length = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images=images.to(device=device)
            labels=labels.to(device=device)
            outputs = model(images)
            loss=LossFun(outputs,labels)
            num_loss+=loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            total_length += predicted.size(0)
    accuracy = correct / total
    loss=num_loss/total_length
    model.train()
    return accuracy,loss

In [ ]:
batch_size = 32
epochs = 10
eta = 1e-4
activationFun='relu'
aug = 'no'
# NUM_OF_CLASSES = 10
# strategy = 2
train_model(epochs,activationFun,aug,batch_size,eta)

100%|██████████| 250/250 [01:46<00:00,  2.34it/s]


0.903487935991999 0.010626074913546866
0.7455 0.024492078840732574


100%|██████████| 250/250 [01:47<00:00,  2.32it/s]


0.962745343167896 0.004169447331687319
0.761 0.0245922172665596


100%|██████████| 250/250 [01:47<00:00,  2.33it/s]


...

0.9957494686835855 0.000586515538789032
0.7655 0.033717502281069754


100%|██████████| 250/250 [01:45<00:00,  2.36it/s]


0.984748093511689 0.001501276881335291
0.7295 0.036536981359124185


100%|██████████| 250/250 [01:46<00:00,  2.35it/s]


0.995624453056632 0.0005332565798837743
0.761 0.03131941863894463


In [ ]:
def main_fun():
    wandb.init(project ='DL_Assignment_2B')
    params = wandb.config
    with wandb.init(project = 'DL_Assignment_2B', name='strategy'+str(params.strategy)+'batchSize'+str(params.batchSize)+'num_epochs'+str(params.num_epochs)+'learning_rate'+str(params.learning_rate)) as run:
        train_model(params.num_epochs,params.aug,params.batchSize,params.learning_rate)
sweep_params = {
    'method' : 'bayes',
    'name'   : 'Accuracy',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'val_accuracy',
    },
    'parameters' : {
        'batchSize':{'values':[32,64]},
        'num_epochs':{'values':[10,5]},
        'learning_rate':{'values' : [1e-3,1e-4]},
        'aug' :{'values':['no','yes']},
    }
}
sweepId = wandb.sweep(sweep_params,project = 'DL_Assignment_2B')
wandb.agent(sweepId,function =main_fun,count = 2)
wandb.finish()